<a href="https://colab.research.google.com/github/sharjeelnawaz8182/standford_car_classification/blob/main/train_cmmr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import keras
from keras.applications.resnet import ResNet50, ResNet152, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
import argparse
import pandas as pd
import numpy as np
import cv2
import os
from random import randint
from tqdm import tqdm
import imgaug as ia
 
img_width, img_height = 224, 224
num_channels = 3
train_data = '/content/drive/MyDrive/compitition/res_net/data/train'
valid_data = '/content/drive/MyDrive/compitition/res_net/data/validation'
num_classes = 196
num_train_samples = 6515
num_valid_samples = 1629
verbose = 1
batch_size = 16  #409 iteration, 16 batch images, at every epoch 16 images will pass and iterate back and forth 409 times updating weights
num_epochs = 100000
patience = 40
 
def random_resize_crop(img, tmp_size, dst_size):
    img = cv2.resize(img, (tmp_size, tmp_size), interpolation = cv2.INTER_CUBIC)
    x = randint(0, tmp_size - dst_size)
    y = randint(0, tmp_size - dst_size)
    crop_img = img[y:y+dst_size, x:x+dst_size]
    return crop_img
def random_eraser(img, p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255):
    img_h, img_w, _ = img.shape
    p_1 = np.random.rand()
    if p_1 > p:
        return img
    while True:
        s = np.random.uniform(s_l, s_h) * img_h * img_w
        r = np.random.uniform(r_1, r_2)
        w = int(np.sqrt(s / r))
        h = int(np.sqrt(s * r))
        left = np.random.randint(0, img_w)
        top = np.random.randint(0, img_h)
        if left + w <= img_w and top + h <= img_h:
            break
    c = np.random.uniform(v_l, v_h)
    img[top:top + h, left:left + w, :] = c
    return img
def pre_process_fun(img):
  tmp_size=280
  dst_size=224
  imge=random_resize_crop(img, tmp_size, dst_size)
  return_img=random_eraser(img=imge, p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255)
  return return_img
model=ResNet50(include_top=True, weights=None,input_shape=(img_width, img_height,num_channels),classes=num_classes)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
trained_models_path = '/content/drive/MyDrive/compitition/res_net/cp1.ckpt'
model.load_weights(trained_models_path)
# prepare data augmentation configuration
train_data_gen = ImageDataGenerator(rotation_range=20,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    shear_range=0,preprocessing_function=pre_process_fun)
valid_data_gen = ImageDataGenerator()
# callbacks
tensor_board = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True) #save file log with trained data at every epoc 
log_file_path = 'logs/training.log'
csv_logger = CSVLogger(log_file_path, append=False) #collecting training data, append-false to overide existing file 
early_stop = EarlyStopping('val_accuracy', patience=patience) #stop training if valid accuracy donot increase for 40 epochs
reduce_lr = ReduceLROnPlateau('val_accuracy', factor=0.1, patience=int(patience / 4), verbose=1) #reducing learning rate after 10 epochs, factor to reduce learning rate
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5' #generating weight files
model_checkpoint = ModelCheckpoint(trained_models_path, monitor='val_accuracy', verbose=1, save_best_only=True) #save_best_only=True epoch with same name having greater accuracy will store
callbacks = [tensor_board, model_checkpoint, csv_logger, early_stop, reduce_lr] #check accuracy,loss etc on a run to help in performing actions
'''train_generator = MixupImageDataGenerator(generator=train_data_gen,
                                      directory=train_data,
                                      batch_size=batch_size,
                                      img_height=img_height,
                                      img_width=img_width,
                                      subset='training')'''
# generators
train_generator = train_data_gen.flow_from_directory(train_data, (img_width, img_height), batch_size=batch_size,
                                                      class_mode='categorical') #flow from directory used when we have subdirectories inside a folder, categorical-for more than two classes
valid_generator = valid_data_gen.flow_from_directory(valid_data, (img_width, img_height), batch_size=batch_size,
                                                      class_mode='categorical')

# fine tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=407,
    validation_data=valid_generator,
    validation_steps=num_valid_samples / batch_size,
    epochs=num_epochs,
    callbacks=callbacks,
    verbose=1)

In [ ]:
train_data